<a href="https://colab.research.google.com/github/biasnhbi/Easy-Hexo/blob/master/DreamBooth_Stable_Diffusion_(NovelAILeaks_Ver_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


In [ ]:
#@title Install Requirements
from IPython.display import clear_output
def cprint(msg):
  clear_output()
  print(msg)


# !curl -LO https://github.com/huggingface/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
# use memory-optim version
!curl -Lo train_dreambooth.py https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/dreambooth.py
!pip install -U pip

%pip install -qq git+https://github.com/huggingface/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio
%pip install omegaconf einops pytorch_lightning 

#@title ### Install xformers from precompiled wheel.
from subprocess import getoutput
from IPython.display import HTML
from IPython.display import clear_output
import time

s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

while True:
    try: 
        gpu=='T4'or gpu=='P100'or gpu=='V100'or gpu=='A100'
        break
    except:
        pass
    print('[1;31mit seems that your GPU is not supported at the moment')
    time.sleep(5)

if (gpu=='T4'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
  
elif (gpu=='P100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='V100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='A100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl  

cprint("Install done.")

Install done.


In [ ]:
#@title Download `animefull` model and traindata (example, for nahida)

#@markdown Path of model
URL = "https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-pruned.tar" #@param {type:"string"}

#@markdown Path of the initial model folder.
SRC_PATH = "/content/animefull-pruned" #@param {type:"string"}
!mkdir -p $SRC_PATH

%cd $SRC_PATH
!apt install -y -qq aria2 > /dev/null
!aria2c --summary-interval=5 -x 6 --allow-overwrite=true -o data.tar $URL
!echo "Decompressing.."
!tar xf data.tar 
!echo "Done"

#@markdown Path of the convert output (as train model input).
MODEL_NAME = "/content/animefull" #@param {type:"string"}

%cd /content
!curl -Lo convert.py https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/convert.py
!python convert.py --checkpoint_path $SRC_PATH/model.ckpt --original_config_file $SRC_PATH/config.yaml --dump_path $MODEL_NAME --scheduler_type ddim

cprint("Convert done.")

download_example_traindata = True #@param {type:"boolean"}
if download_example_traindata:
  !mkdir -p nahida
  !curl -L https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/train-nahida.tar | tar x -C nahida
  !rm -fr nahida/._train*

cprint('DONE !')

DONE !


In [ ]:
#@title Settings

#@markdown Path for images of the concept for training.
INSTANCE_PROMPT = "nahida" #@param {type:"string"}
INSTANCE_DIR = "/content/nahida" #@param {type:"string"}
!mkdir -p $INSTANCE_DIR

#@markdown A general name for class like dog for dog images.
CLASS_PROMPT = "full body illustration of a anime cute face young girl in dramatic light, masterpiece and best quality." #@param {type:"string"}
CLASS_DIR = "/content/nahida_autogen" #@param {type:"string"}


#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = False #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown Enter the directory name to save model at.

OUTPUT_DIR = "remote/nahida-dreambooth-2" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

[*] Weights will be saved at /content/remote/nahida-dreambooth-2


## Start Training

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


In [ ]:
%cd /content
!mkdir -p $OUTPUT_DIR
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="{INSTANCE_PROMPT}" \
  --class_prompt="{CLASS_PROMPT}" \
  --seed=1337 \
  --resolution=482 \
  --mixed_precision="fp16" \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="cosine_with_restarts" \
  --lr_warmup_steps=100 \
  --num_class_images=150 \
  --sample_batch_size=4 \
  --max_train_steps=2000 \
  --save_interval=500 \
  --gradient_checkpointing \
  --use_8bit_adam 

# disabled: --save_data_every=50 --save_img_every=50 --not_cache_latents 

/content
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_cpu_threads_per_process` was set to `1` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2022-10-24 00:15:00,688] [WARNING] [runner.py:179:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2022-10-24 00:15:00,719] [INFO] [runner.py:507:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --no_local_rank train_dreambooth.py --pretrained_model_name_or_path=/content/animefull --instance_data_dir=/content/nahida --class_data_dir=/content/nahida_autogen --output_dir=/content/remote/nahida-dreambooth-2 --with_prior_preservation --prior_loss_weight=1.0 --instance_prompt=nahida --class_prompt=full body illustration of a anime cute face young girl in dramatic light, 

Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

## Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

In [ ]:
!curl -Lo back_convert.py https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/back_convert.py

#@markdown Run conversion.
ckpt_path = OUTPUT_DIR + "/model.ckpt"
src_ckpt_path = SRC_PATH + "/model.ckpt"

!python back_convert.py --model_path $OUTPUT_DIR  --src_path $src_ckpt_path --checkpoint_path $ckpt_path 
print(f"[*] Converted ckpt saved at {ckpt_path}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 87516  100 87516    0     0   143k      0 --:--:-- --:--:-- --:--:--  143k
[*] Converted ckpt saved at /content/remote/nahida-dreambooth-2/model.ckpt


## Inference

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

#@markdown Run Gradio UI for generating images.
model_path = OUTPUT_DIR  #@param {type:"string"}         

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
g_cuda = None


import gradio as gr

def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of sks guy, digital painting")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=50)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)

KeyboardInterrupt: ignored